<a href="https://colab.research.google.com/github/fbison/PrevisaoTemperatura/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

def read_file(filename):
  with open(filename, 'r') as f:
    lines = f.readlines()
  return lines

def read_numbers(lines):
  # Converte as linhas para um array de números de ponto flutuante
  numbers = []
  for line in lines:
    # Remove espaços em branco e converte cada número para float com duas casas decimais
    numbers.extend([round(float(num), 2) for num in line.strip().split()])
  return numbers

# Upload the file to Colab
from google.colab import files
uploaded = files.upload()

# Read the lines from the uploaded file
xTrain = read_numbers(read_file('Xtrain.txt'))
xTest = read_numbers(read_file('Xtest.txt'))
print(xTrain)

Saving Xtest.txt to Xtest (2).txt
Saving Xtrain.txt to Xtrain (3).txt
[5727.99, 5635.07, 5599.42, 5567.89, 5530.23, 5457.2, 5413.43, 5532.47, 5503.79, 5634.86, 5804.85, 5834.6, 5817.14, 5823.04, 6053.14, 6201.81, 6082.41, 6227.16, 6181.59, 6043.89, 6156.19, 6245.51, 6355.85, 6432.1, 6286.46, 6338.78, 6487.97, 6444.6, 6508.52, 6652.19, 6432.91, 6293.45, 6214.68, 6188.8, 6411.65, 6299.14, 6222.59, 6050.99, 6096.31, 5968.46, 5909.39, 6143.34, 6123.29, 6277.31, 6363.58, 6244.15, 6130.75, 6165.26, 6257.25, 6222.76, 6304.19, 6227.71, 6227.1, 6348.41, 6352.86, 6292.14, 6337.13, 6314.7, 6248.95, 6202.71, 6028.06, 6126.1, 6294.76, 6270.03, 6259.4, 6232.55, 6270.04, 6276.87, 6254.01, 6334.62, 6382.76, 6415.78, 6358.07, 6390.98, 6464.29, 6534.28, 6512.41, 6489.64, 6560.21, 6588.26, 6551.74, 6563.79, 6513.88, 6535.92, 6458.88, 6446.87, 6521.78, 6605.81, 6626.94, 6612.66, 6651.58, 6631.08, 6626.75, 6608.68, 6631.32, 6744.94, 6733.54, 6766.04, 6798.15, 6798.92, 6675.62, 6648.61, 6588.85, 6503.36, 64

In [22]:
import numpy as np

def create_matrix(xTrain, w, h):
    matrix = [xTrain[i:i + w] for i in range(len(xTrain) - w - h + 1)]
    return matrix

def distance_vector(matrixTrain, lineMatrixTest):
    # Calcula a distância entre `lineMatrixTest` e cada linha em `matrixTrain`
    distances = [np.sum((np.array(lineTrain) - np.array(lineMatrixTest)) ** 2) for lineTrain in matrixTrain]
    return distances
def create_YTest(matrixTrain, matrixTest, Ytrain, k):
    YTest = []
    for lineMatrixTest in matrixTest:
        # Calcula as distâncias para cada linha de matrixTrain
        distances = distance_vector(matrixTrain, lineMatrixTest)

        # Encontra os índices das k menores distâncias
        nearest_indices = np.argsort(distances)[:k]

        # Seleciona os valores correspondentes em Ytrain e calcula a média
        nearest_values = [Ytrain[i] for i in nearest_indices]
        YTest.append(np.mean(nearest_values))

    return YTest
def mean_absolute_error(y_true, y_pred):
    min_length = min(len(y_true), len(y_pred))
    y_true = y_true[:min_length]
    y_pred = y_pred[:min_length]
    return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))

def optimize_parameters(xTrain, xTest, k_values, w_values, h_values):
    best_error = float('inf')
    best_params = {'k': None, 'w': None, 'h': None}

    # Iterar sobre todos os valores de k, w e h
    for k in k_values:
        for w in w_values:
            for h in h_values:
                # Criar as matrizes com os parâmetros atuais
                matrixTrain = create_matrix(xTrain, w, h)
                matrixTest = create_matrix(xTest, w, h)

                # Criar yTrain e calcular yTest com os parâmetros atuais
                yTrain = [xTrain[i + w + h] for i in range(len(xTrain) - w - h)]
                yAnswer = [xTrain[i + w + h] for i in range(len(xTrain) - w - h)]
                yTest = create_YTest(matrixTrain, matrixTest, yTrain, k)

                # Calcular o erro
                error = mean_absolute_error(yAnswer, yTest)

                # Atualizar os melhores parâmetros se o erro atual for menor
                if error < best_error:
                    best_error = error
                    best_params = {'k': k, 'w': w, 'h': h}
                    print(f"Novo melhor erro encontrado: {best_error} com k={k}, w={w}, h={h}")

    return best_params, best_error

In [24]:
k_values = range(1, 200)
w_values = range(2, 200)
h_values = range(1, 200)

# Otimizar parâmetros
best_params, best_error = optimize_parameters(xTrain, xTest, k_values, w_values, h_values)
print("Melhores parâmetros encontrados:", best_params)
print("Melhor erro encontrado:", best_error)

Novo melhor erro encontrado: 3462.8013655462187 com k=1, w=2, h=1
Novo melhor erro encontrado: 2610.518589473684 com k=1, w=2, h=2
Novo melhor erro encontrado: 2358.8186864406784 com k=1, w=2, h=5
Novo melhor erro encontrado: 2305.727190265487 com k=1, w=2, h=25
Novo melhor erro encontrado: 2301.436186252771 com k=1, w=2, h=26
Novo melhor erro encontrado: 2238.189533333333 com k=1, w=2, h=27
Novo melhor erro encontrado: 2096.145456570156 com k=1, w=2, h=28
Novo melhor erro encontrado: 1982.7157466063347 com k=1, w=2, h=35
Novo melhor erro encontrado: 1864.943605442177 com k=1, w=2, h=36
Novo melhor erro encontrado: 1829.2762437810943 com k=1, w=2, h=75
Novo melhor erro encontrado: 1661.4043890274313 com k=1, w=2, h=76
Novo melhor erro encontrado: 1653.4172932330828 com k=1, w=2, h=78
Novo melhor erro encontrado: 1639.534116161616 com k=1, w=2, h=81
Novo melhor erro encontrado: 1628.969467005076 com k=1, w=2, h=83
Novo melhor erro encontrado: 1598.5106084656086 com k=1, w=2, h=99
Novo m

KeyboardInterrupt: 